# Microsoft Purview Unified Catalog - PowerShell Integration Examples

This notebook demonstrates how to use the PVW CLI with PowerShell scripting for automation and data manipulation.

## What You'll Learn
- Fetching terms as JSON and converting to PowerShell objects
- Filtering and searching through term collections
- Bulk operations using PowerShell loops
- Exporting data to CSV for reporting
- Practical automation scenarios

## Prerequisites
- PVW CLI installed (`pip install pvw-cli`)
- Azure authentication configured (`az login`)
- PowerShell kernel for Jupyter (or run cells in PowerShell terminal)
- At least one governance domain with terms created

## Setup and Configuration

In [ ]:
# Set your domain ID here
$domainId = "<your-domain-id>"  # Replace with your actual domain ID

Write-Host "Configuration loaded. Using domain ID: $domainId" -ForegroundColor Green

## Example 1: Fetch and Display All Terms

Get all terms from a domain and convert to PowerShell objects.

In [ ]:
# Fetch all terms as JSON and convert to PowerShell objects
$terms = py -m purviewcli uc term list --domain-id $domainId --output json | ConvertFrom-Json

# Display summary
Write-Host "\nFound $($terms.Count) terms in domain" -ForegroundColor Cyan

# Display first 5 terms
$terms | Select-Object -First 5 | Format-Table name, status, id -AutoSize

## Example 2: Find a Specific Term by Name

Search for a term by its exact name.

In [ ]:
# Find term by exact name
$searchName = "Customer Acquisition Cost"
$term = $terms | Where-Object { $_.name -eq $searchName }

if ($term) {
    Write-Host "\nFound term: $($term.name)" -ForegroundColor Green
    Write-Host "ID: $($term.id)"
    Write-Host "Status: $($term.status)"
    Write-Host "Description: $($term.description)"
    if ($term.acronyms) {
        Write-Host "Acronyms: $($term.acronyms -join ', ')"
    }
} else {
    Write-Host "Term '$searchName' not found" -ForegroundColor Yellow
}

## Example 3: Search Terms by Partial Name

Find all terms containing a specific keyword.

In [ ]:
# Search for terms containing a keyword
$keyword = "Customer"
$matches = $terms | Where-Object { $_.name -like "*$keyword*" }

Write-Host "\nFound $($matches.Count) terms containing '$keyword'" -ForegroundColor Cyan
$matches | Format-Table name, status, id -AutoSize

## Example 4: Filter Terms by Status

Get all terms with a specific status (Published, Draft, Archived).

In [ ]:
# Get all published terms
$publishedTerms = $terms | Where-Object { $_.status -eq "Published" }
Write-Host "\nPublished Terms: $($publishedTerms.Count)" -ForegroundColor Green
$publishedTerms | Format-Table name, status -AutoSize

# Get all draft terms
$draftTerms = $terms | Where-Object { $_.status -eq "Draft" }
Write-Host "\nDraft Terms: $($draftTerms.Count)" -ForegroundColor Yellow
$draftTerms | Format-Table name, status -AutoSize

## Example 5: Search in Descriptions

Find terms based on keywords in their descriptions.

In [ ]:
# Search for terms with specific keywords in description
$descKeyword = "customer"
$matches = $terms | Where-Object { $_.description -like "*$descKeyword*" }

Write-Host "\nTerms with '$descKeyword' in description: $($matches.Count)" -ForegroundColor Cyan
$matches | Select-Object name, @{Name='Description';Expression={$_.description.Substring(0,[Math]::Min(50,$_.description.Length)) + '...'}} | Format-Table -AutoSize

## Example 6: Find Terms by Acronym

Search for terms that have a specific acronym.

In [ ]:
# Find terms with a specific acronym
$acronymSearch = "CAC"
$matches = $terms | Where-Object { $_.acronyms -contains $acronymSearch }

Write-Host "\nTerms with acronym '$acronymSearch': $($matches.Count)" -ForegroundColor Cyan
$matches | Format-Table name, @{Name='Acronyms';Expression={$_.acronyms -join ', '}}, status -AutoSize

## Example 7: Get Term Details

Fetch detailed information about a specific term.

In [ ]:
# Get the first term's ID
$termId = $terms[0].id

# Fetch detailed information
$termDetails = py -m purviewcli uc term show --term-id $termId --json | ConvertFrom-Json

Write-Host "\nTerm Details:" -ForegroundColor Cyan
Write-Host "Name: $($termDetails.name)"
Write-Host "ID: $($termDetails.id)"
Write-Host "Status: $($termDetails.status)"
Write-Host "Description: $($termDetails.description)"

if ($termDetails.acronyms) {
    Write-Host "Acronyms: $($termDetails.acronyms -join ', ')"
}

if ($termDetails.contacts.owner) {
    Write-Host "\nOwners:"
    $termDetails.contacts.owner | ForEach-Object { Write-Host "  - $($_.id)" }
}

if ($termDetails.resources) {
    Write-Host "\nResources:"
    $termDetails.resources | ForEach-Object { Write-Host "  - $($_.name): $($_.url)" }
}

## Example 8: Export Terms to CSV

Export term data to a CSV file for reporting or analysis.

In [ ]:
# Export terms to CSV
$exportPath = "../csv/terms_export.csv"

# Create export data with flattened structure
$exportData = $terms | ForEach-Object {
    [PSCustomObject]@{
        Name = $_.name
        ID = $_.id
        Status = $_.status
        Description = $_.description
        Acronyms = if ($_.acronyms) { $_.acronyms -join '; ' } else { '' }
        Domain = $_.domain
    }
}

$exportData | Export-Csv -Path $exportPath -NoTypeInformation -Encoding UTF8

Write-Host "\nExported $($exportData.Count) terms to: $exportPath" -ForegroundColor Green

## Example 9: Group Terms by Status

Get statistics about term status distribution.

In [ ]:
# Group terms by status and count
$statusGroups = $terms | Group-Object -Property status

Write-Host "\nTerm Status Distribution:" -ForegroundColor Cyan
$statusGroups | ForEach-Object {
    Write-Host "  $($_.Name): $($_.Count) terms" -ForegroundColor $(if ($_.Name -eq 'Published') { 'Green' } else { 'Yellow' })
}

# Display as table
$statusGroups | Select-Object @{Name='Status';Expression={$_.Name}}, Count | Format-Table -AutoSize

## Example 10: Find Terms Without Acronyms

Identify terms that might need acronyms added.

In [ ]:
# Find terms without acronyms
$termsWithoutAcronyms = $terms | Where-Object { -not $_.acronyms -or $_.acronyms.Count -eq 0 }

Write-Host "\nTerms without acronyms: $($termsWithoutAcronyms.Count)" -ForegroundColor Yellow
$termsWithoutAcronyms | Select-Object name, status, id | Format-Table -AutoSize

## Example 11: Bulk Update - Change Status

Update multiple terms from Draft to Published.

In [ ]:
# Get all draft terms
$draftTerms = $terms | Where-Object { $_.status -eq "Draft" }

Write-Host "\nFound $($draftTerms.Count) draft terms" -ForegroundColor Yellow
Write-Host "\n[INFO] Uncomment the code below to publish all draft terms\n" -ForegroundColor Cyan

# Uncomment to execute bulk publish
# $confirmPublish = Read-Host "Do you want to publish all draft terms? (yes/no)"
# if ($confirmPublish -eq "yes") {
#     $publishedCount = 0
#     $failedCount = 0
#     
#     foreach ($term in $draftTerms) {
#         Write-Host "Publishing: $($term.name)..." -NoNewline
#         try {
#             py -m purviewcli uc term update --term-id $term.id --status Published
#             Write-Host " [OK]" -ForegroundColor Green
#             $publishedCount++
#         } catch {
#             Write-Host " [FAILED]" -ForegroundColor Red
#             $failedCount++
#         }
#         Start-Sleep -Milliseconds 200  # Rate limiting
#     }
#     
#     Write-Host "\nPublished: $publishedCount, Failed: $failedCount" -ForegroundColor Cyan
# }

Write-Host "Preview of terms that would be published:"
$draftTerms | Select-Object -First 5 | Format-Table name, status -AutoSize

## Example 12: List All Domains

Fetch all governance domains and their details.

In [ ]:
# List all governance domains
$domains = py -m purviewcli uc domain list --output json | ConvertFrom-Json

Write-Host "\nFound $($domains.Count) governance domains" -ForegroundColor Cyan
$domains | Format-Table id, name, type, status -AutoSize

# Show domain IDs for easy reference
Write-Host "\nDomain IDs for reference:" -ForegroundColor Yellow
$domains | ForEach-Object {
    Write-Host "  $($_.name): $($_.id)" -ForegroundColor Gray
}

## Example 13: Compare Terms Across Domains

Get term counts for multiple domains.

In [ ]:
# Get all domains
$domains = py -m purviewcli uc domain list --output json | ConvertFrom-Json

Write-Host "\nTerm counts by domain:" -ForegroundColor Cyan

$domainStats = @()
foreach ($domain in $domains) {
    $domainTerms = py -m purviewcli uc term list --domain-id $domain.id --output json | ConvertFrom-Json
    $publishedCount = ($domainTerms | Where-Object { $_.status -eq "Published" }).Count
    $draftCount = ($domainTerms | Where-Object { $_.status -eq "Draft" }).Count
    
    $domainStats += [PSCustomObject]@{
        Domain = $domain.name
        Total = $domainTerms.Count
        Published = $publishedCount
        Draft = $draftCount
    }
}

$domainStats | Format-Table -AutoSize

## Example 14: Search Terms with Multiple Criteria

Combine multiple filters for advanced searches.

In [ ]:
# Search for published terms containing "customer" in name or description
$complexSearch = $terms | Where-Object {
    ($_.status -eq "Published") -and 
    (($_.name -like "*customer*") -or ($_.description -like "*customer*"))
}

Write-Host "\nPublished terms related to 'customer': $($complexSearch.Count)" -ForegroundColor Cyan
$complexSearch | Format-Table name, status, @{Name='HasAcronym';Expression={$_.acronyms.Count -gt 0}} -AutoSize

## Example 15: Create a Term Report

Generate a comprehensive report with term statistics.

In [ ]:
# Generate comprehensive term report
Write-Host "\n" + "="*80 -ForegroundColor Cyan
Write-Host "UNIFIED CATALOG TERMS REPORT" -ForegroundColor Cyan
Write-Host "="*80 + "\n" -ForegroundColor Cyan

Write-Host "Domain ID: $domainId" -ForegroundColor Gray
Write-Host "Generated: $(Get-Date -Format 'yyyy-MM-dd HH:mm:ss')" -ForegroundColor Gray
Write-Host ""

# Overall stats
Write-Host "OVERVIEW" -ForegroundColor Yellow
Write-Host "Total Terms: $($terms.Count)"
Write-Host "Published: $($terms | Where-Object { $_.status -eq 'Published' }).Count"
Write-Host "Draft: $($terms | Where-Object { $_.status -eq 'Draft' }).Count"
Write-Host "Archived: $($terms | Where-Object { $_.status -eq 'Archived' }).Count"
Write-Host ""

# Acronym stats
$withAcronyms = ($terms | Where-Object { $_.acronyms -and $_.acronyms.Count -gt 0 }).Count
Write-Host "ACRONYMS" -ForegroundColor Yellow
Write-Host "Terms with acronyms: $withAcronyms"
Write-Host "Terms without acronyms: $($terms.Count - $withAcronyms)"
Write-Host ""

# Top 10 terms
Write-Host "TOP 10 TERMS (by name)" -ForegroundColor Yellow
$terms | Sort-Object name | Select-Object -First 10 | Format-Table name, status -AutoSize

Write-Host "\n" + "="*80 -ForegroundColor Cyan

## Summary

This notebook demonstrated:

1. ✅ **Fetching data** - Using `--output json` for PowerShell integration
2. ✅ **Filtering** - Using `Where-Object` for various search criteria
3. ✅ **Searching** - By name, description, status, and acronyms
4. ✅ **Exporting** - Creating CSV reports for external analysis
5. ✅ **Statistics** - Grouping and counting terms
6. ✅ **Bulk operations** - Updating multiple terms efficiently
7. ✅ **Reporting** - Generating comprehensive term reports

### Key PowerShell Cmdlets Used

- `ConvertFrom-Json` - Parse JSON output from CLI
- `Where-Object` - Filter collections
- `Select-Object` - Choose specific properties
- `Format-Table` - Display data in tables
- `Group-Object` - Group and count items
- `Export-Csv` - Export data to CSV files
- `ForEach-Object` - Loop through collections

### Next Steps

- Automate term creation from external data sources
- Build approval workflows for term publishing
- Create scheduled reports using PowerShell scripts
- Integrate with Azure DevOps or GitHub Actions
- Build custom dashboards with term statistics

For more examples, see:
- `unified_catalog_terms_examples.ipynb` - Python examples
- `scripts/delete-all-uc-terms.ps1` - Bulk delete script
- `README.md` - Full documentation